In [175]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [176]:
path_train_input=[]
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0372_07_00/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0372_07_01/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0374_11_00/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0374_11_01/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0374_11_02/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0374_11_03/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0378_13_00/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0379_11_00/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0380_11_00/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0384_11_01/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0384_11_02/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0384_11_03/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0384_11_04/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0385_11_00/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0386_11_00/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0477_11_00/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0857_11_00/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0868_11_01/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0868_11_02/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0871_11_01/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0881_11_00/*.png')
path_train_input.append('E:/gopro/GOPRO_Large_all/train/GOPR0884_11_00/*.png')

print(len(path_train_input))

22


In [177]:
path_test_input=[]
path_test_input.append('E:/gopro/GOPRO_Large_all/test/GOPR0384_11_00/*.png')
path_test_input.append('E:/gopro/GOPRO_Large_all/test/GOPR0384_11_05/*.png')
path_test_input.append('E:/gopro/GOPRO_Large_all/test/GOPR0385_11_01/*.png')
path_test_input.append('E:/gopro/GOPRO_Large_all/test/GOPR0396_11_00/*.png')
path_test_input.append('E:/gopro/GOPRO_Large_all/test/GOPR0410_11_00/*.png')
path_test_input.append('E:/gopro/GOPRO_Large_all/test/GOPR0854_11_00/*.png')
path_test_input.append('E:/gopro/GOPRO_Large_all/test/GOPR0862_11_00/*.png')
path_test_input.append('E:/gopro/GOPRO_Large_all/test/GOPR0868_11_00/*.png')
path_test_input.append('E:/gopro/GOPRO_Large_all/test/GOPR0869_11_00/*.png')
path_test_input.append('E:/gopro/GOPRO_Large_all/test/GOPR0871_11_00/*.png')
path_test_input.append('E:/gopro/GOPRO_Large_all/test/GOPR0881_11_01/*.png')



print(len(path_test_input))

11


In [178]:
for name in path_train_input:
    filename=name[0:-6]
    filename=filename.replace("GOPRO_Large_all","BLE_raw_bi_large")
    if os.path.exists(filename) is not True:
        os.mkdir(filename)
for name in path_test_input:
    filename=name[0:-6]
    filename=filename.replace("GOPRO_Large_all","BLE_raw_bi_large")
    if os.path.exists(filename) is not True:
        os.mkdir(filename)

In [179]:
for name in path_train_input:
    filename=name[0:-6]
    filename=filename.replace("GOPRO_Large_all/train","BLE_all/bl_train")
    if os.path.exists(filename) is not True:
        os.mkdir(filename)
for name in path_test_input:
    filename=name[0:-6]
    filename=filename.replace("GOPRO_Large_all/test","BLE_all/bl_test")
    if os.path.exists(filename) is not True:
        os.mkdir(filename)

In [ ]:
import numpy
from scipy.interpolate import InterpolatedUnivariateSpline 
crf_path = 'crf.npy'
device='cpu'
C=3
H=720
W=1280
crf_inv = np.load('crf.npy').squeeze()

# 1st order approximation at RGB=250 to regularize extreme responses at RGB>250
diff = (crf_inv[251] - crf_inv[249])/2
for i in range(251, 256):
    crf_inv[i] = crf_inv[i-1] + diff

#crf_inv[:,0]=crf_inv[:,1]
#crf_inv[:,2]=crf_inv[:,1]
x=np.linspace(0,255,256)
y=crf_inv[:,0]
f1=InterpolatedUnivariateSpline(x,y)
f2=InterpolatedUnivariateSpline(y,x)
# frame interpolation, etc.
path=path_train_input
for i in range(len(path)): #len(path)
    print('now processing:',i)
    imglist=glob.glob(path[i])
    imglist.sort()
    imgsignal_list=[]
    print('buffering')
    for k in range(len(imglist)):#len(imglist)
        img=Image.open(imglist[k])
        img=np.array(img)
        img=img.reshape(-1)
        img_signal=f1(img)
        imgsignal_list.append(img_signal)
    print('finish buffering')
    for j in range(len(imglist)-7):
        
        
        img1_signal=imgsignal_list[j]
        img2_signal=imgsignal_list[j+1]
        img3_signal=imgsignal_list[j+2]
        img4_signal=imgsignal_list[j+3]
        img5_signal=imgsignal_list[j+4]
        img6_signal=imgsignal_list[j+5]
        img7_signal=imgsignal_list[j+6]
        img8_signal=imgsignal_list[j+7]
        
        img_blur_signal=(img1_signal+img2_signal+img3_signal+img4_signal
                 +img5_signal+img6_signal+img7_signal+img8_signal)/8.0
        img_blur=f2(img_blur_signal)
        img_blur=img_blur.reshape(H,W,C)
        
        outputname=imglist[j].replace("GOPRO_Large_all","BLE_all")
        img_blur=Image.fromarray(img_blur.astype('uint8'))
        img_blur.save(outputname)
        
   



plt.figure()
plt.imshow(img_blur)

now processing: 0
buffering
finish buffering
now processing: 1
buffering
finish buffering
now processing: 2
buffering
finish buffering
now processing: 3
buffering
finish buffering
now processing: 4
buffering
finish buffering
now processing: 5
buffering
finish buffering
now processing: 6
buffering
finish buffering
now processing: 7
buffering
finish buffering
now processing: 8
buffering
finish buffering
now processing: 9
buffering
finish buffering
now processing: 10
buffering
finish buffering


In [ ]:
import numpy
from scipy.interpolate import InterpolatedUnivariateSpline 
crf_path = 'crf.npy'
device='cpu'
C=3
H=720
W=1280
crf_inv = np.load('crf.npy').squeeze()

# 1st order approximation at RGB=250 to regularize extreme responses at RGB>250
diff = (crf_inv[251] - crf_inv[249])/2
for i in range(251, 256):
    crf_inv[i] = crf_inv[i-1] + diff

#crf_inv[:,0]=crf_inv[:,1]
#crf_inv[:,2]=crf_inv[:,1]
x=np.linspace(0,255,256)
y=crf_inv[:,0]
f1=InterpolatedUnivariateSpline(x,y)
f2=InterpolatedUnivariateSpline(y,x)
# frame interpolation, etc.
path=path_test_input
for i in range(len(path)): #len(path)
    print('now processing:',i)
    imglist=glob.glob(path[i])
    imglist.sort()
    imgsignal_list=[]
    print('buffering')
    for k in range(len(imglist)):#len(imglist)
        img=Image.open(imglist[k])
        img=np.array(img)
        img=img.reshape(-1)
        img_signal=f1(img)
        imgsignal_list.append(img_signal)
    print('finish buffering')
    for j in range(len(imglist)-7):
        
        
        img1_signal=imgsignal_list[j]
        img2_signal=imgsignal_list[j+1]
        img3_signal=imgsignal_list[j+2]
        img4_signal=imgsignal_list[j+3]
        img5_signal=imgsignal_list[j+4]
        img6_signal=imgsignal_list[j+5]
        img7_signal=imgsignal_list[j+6]
        img8_signal=imgsignal_list[j+7]
        
        img_blur_signal=(img1_signal+img2_signal+img3_signal+img4_signal
                 +img5_signal+img6_signal+img7_signal+img8_signal)/8.0
        img_blur=f2(img_blur_signal)
        img_blur=img_blur.reshape(H,W,C)
        
        outputname=imglist[j].replace("GOPRO_Large_all","BLE_all")
        img_blur=Image.fromarray(img_blur.astype('uint8'))
        img_blur.save(outputname)
        
   



plt.figure()
plt.imshow(img_blur)

In [ ]:
import numpy
from scipy.interpolate import InterpolatedUnivariateSpline 
crf_path = 'crf.npy'
device='cpu'
C=3
H=720
W=1280
crf_inv = np.load('crf.npy').squeeze()

# 1st order approximation at RGB=250 to regularize extreme responses at RGB>250
diff = (crf_inv[251] - crf_inv[249])/2
for i in range(251, 256):
    crf_inv[i] = crf_inv[i-1] + diff

#crf_inv[:,0]=crf_inv[:,1]
#crf_inv[:,2]=crf_inv[:,1]
x=np.linspace(0,255,256)
y=crf_inv[:,0]
f1=InterpolatedUnivariateSpline(x,y)
f2=InterpolatedUnivariateSpline(y,x)
# frame interpolation, etc.
path=path_train_input
for i in range(len(path)):
    print('now processing:',i)
    imglist=glob.glob(path[i])
    imglist.sort()
    imgsignal_list=[]
    print('buffering')
    for k in range(len(imglist)):
        img=Image.open(imglist[k])
        img=np.array(img)
        img=img.reshape(-1)
        img_signal=f1(img)
        imgsignal_list.append(img_signal)
    print('finish buffering')
    for j in range(len(imglist)-7):
        img1=Image.open(imglist[j])
        img2=Image.open(imglist[j+1])
        img3=Image.open(imglist[j+2])
        img4=Image.open(imglist[j+3])
        img5=Image.open(imglist[j+4])
        img6=Image.open(imglist[j+5])
        img7=Image.open(imglist[j+6])
        img8=Image.open(imglist[j+7])
        
        img1=np.array(img1)
        img2=np.array(img2)
        img3=np.array(img3)
        img4=np.array(img4)
        img5=np.array(img5)
        img6=np.array(img6)
        img7=np.array(img7)
        img8=np.array(img8)
        
        img1=img1.reshape(-1)
        img2=img2.reshape(-1)
        img3=img3.reshape(-1)
        img4=img4.reshape(-1)
        img5=img5.reshape(-1)
        img6=img6.reshape(-1)
        img7=img7.reshape(-1)
        img8=img8.reshape(-1)
        
        img1_signal=f1(img1)
        img2_signal=f1(img2)
        img3_signal=f1(img3)
        img4_signal=f1(img4)
        img5_signal=f1(img5)
        img6_signal=f1(img6)
        img7_signal=f1(img7)
        img8_signal=f1(img8)
        
        img_blur_signal=(img1_signal+img2_signal+img3_signal+img4_signal
                 +img5_signal+img6_signal+img7_signal+img8_signal)/8.0
        img_blur=f2(img_blur_signal)
        img_blur=img_blur.reshape(H,W,C)
        
    break



plt.figure()
plt.imshow(img_blur/255.0)
print(img_blur.max())

In [19]:
pathlist=[]
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0372_07_00/',7,204]) #7 61
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0372_07_01/',7,117]) #7 59
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0374_11_00/',11,237]) #11 116
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0374_11_01/',11,212]) #11 108
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0374_11_02/',11,278]) #11 154
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0374_11_03/',11,240]) #11 125
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0378_13_00/',13,76]) #13 40
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0379_11_00/',11,94]) #11 67
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0380_11_00/',11,236]) #11 121
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0384_11_01/',11,205]) #11 100
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0384_11_02/',11,130]) #11 62
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0384_11_03/',11,171]) #11 86
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0384_11_04/',11,270]) #11 150
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0385_11_00/',11,96]) #11 64
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0386_11_00/',11,199]) #11 107
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0477_11_00/',11,284]) #11 155
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0857_11_00/',11,111]) #11 54 
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0868_11_01/',11,94]) #11 46
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0868_11_02/',11,129]) #11 64
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0871_11_01/',11,145]) #11 65
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0881_11_00/',11,138]) #11 84
pathlist.append(['E:/gopro/GOPRO_Large_all/train/GOPR0884_11_00/',11,111]) #11 55

In [21]:
import numpy as np
from scipy.interpolate import InterpolatedUnivariateSpline 
import glob
from PIL import Image
import os
import argparse


crf_path = 'crf.npy'
device='cpu'
C=3
H=720
W=1280
crf_inv = np.load('crf.npy').squeeze()

# 1st order approximation at RGB=250 to regularize extreme responses at RGB>250
diff = (crf_inv[251] - crf_inv[249])/2
for i in range(251, 256):
    crf_inv[i] = crf_inv[i-1] + diff

#crf_inv[:,0]=crf_inv[:,1]
#crf_inv[:,2]=crf_inv[:,1]
x=np.linspace(0,255,256)
y=crf_inv[:,0]
f1=InterpolatedUnivariateSpline(x,y)
f2=InterpolatedUnivariateSpline(y,x)
# frame interpolation, etc.

for name in pathlist:
    path = name[0]
    images = glob.glob(os.path.join(path, '*.png')) + \
                glob.glob(os.path.join(path, '*.jpg'))
    imglist = sorted(images)
    frames = name[1]
    imgsignal_list=[]
    print('buffering')
    for k in range(len(imglist)):
        img=Image.open(imglist[k])
        img=np.array(img)
        img=img.reshape(-1)
        img_signal=f1(img)
        imgsignal_list.append(img_signal)
    print('finish buffering')
    for i in range(0,len(imglist)-frames):
        print(i)
        img_blur_signal=0
        for j in range(frames):
            img_signal=imgsignal_list[i+j]
            img_blur_signal += img_signal
        img_blur_signal = img_blur_signal/frames
        img_blur=f2(img_blur_signal)
        img_blur=img_blur.reshape(H,W,C)
        
        outputname=imglist[i].replace("GOPRO_Large_all","BLE_raw_bi_large")
        img_blur=Image.fromarray(img_blur.astype('uint8'))
        img_blur.save(outputname)

buffering
finish buffering
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270

KeyboardInterrupt: 

In [16]:
pathlist=[]
pathlist.append(['E:/gopro/GOPRO_Large_all/test/GOPR0384_11_00/',11])
pathlist.append(['E:/gopro/GOPRO_Large_all/test/GOPR0384_11_05/',11])
pathlist.append(['E:/gopro/GOPRO_Large_all/test/GOPR0385_11_01/',11])
pathlist.append(['E:/gopro/GOPRO_Large_all/test/GOPR0396_11_00/',11])
pathlist.append(['E:/gopro/GOPRO_Large_all/test/GOPR0410_11_00/',11])
pathlist.append(['E:/gopro/GOPRO_Large_all/test/GOPR0854_11_00/',11])
pathlist.append(['E:/gopro/GOPRO_Large_all/test/GOPR0862_11_00/',11])
pathlist.append(['E:/gopro/GOPRO_Large_all/test/GOPR0868_11_00/',11])
pathlist.append(['E:/gopro/GOPRO_Large_all/test/GOPR0869_11_00/',11])
pathlist.append(['E:/gopro/GOPRO_Large_all/test/GOPR0871_11_00/',11])
pathlist.append(['E:/gopro/GOPRO_Large_all/test/GOPR0881_11_01/',11])

In [18]:
import numpy as np
from scipy.interpolate import InterpolatedUnivariateSpline 
import glob
from PIL import Image
import os
import argparse


crf_path = 'crf.npy'
device='cpu'
C=3
H=720
W=1280
crf_inv = np.load('crf.npy').squeeze()

# 1st order approximation at RGB=250 to regularize extreme responses at RGB>250
diff = (crf_inv[251] - crf_inv[249])/2
for i in range(251, 256):
    crf_inv[i] = crf_inv[i-1] + diff

#crf_inv[:,0]=crf_inv[:,1]
#crf_inv[:,2]=crf_inv[:,1]
x=np.linspace(0,255,256)
y=crf_inv[:,0]
f1=InterpolatedUnivariateSpline(x,y)
f2=InterpolatedUnivariateSpline(y,x)
# frame interpolation, etc.

for name in pathlist:
    path = name[0]
    images = glob.glob(os.path.join(path, '*.png')) + \
                glob.glob(os.path.join(path, '*.jpg'))
    imglist = sorted(images)
    frames = name[1]
    imgsignal_list=[]
    print('buffering')
    for k in range(len(imglist)):
        img=Image.open(imglist[k])
        img=np.array(img)
        img=img.reshape(-1)
        img_signal=f1(img)
        imgsignal_list.append(img_signal)
    print('finish buffering')
    for i in range(0,len(images)-frames):
        img_blur_signal=0
        for j in range(frames):
            print(j)
            img_signal=imgsignal_list[i+j]
            img_blur_signal += img_signal
        img_blur_signal = img_blur_signal/frames
        img_blur=f2(img_blur_signal)
        img_blur=img_blur.reshape(H,W,C)
        
        outputname=imglist[i].replace("GOPRO_Large_all","BLE_raw_bi_large")
        img_blur=Image.fromarray(img_blur.astype('uint8'))
        img_blur.save(outputname)

buffering
finish buffering
buffering
finish buffering
buffering
finish buffering
buffering
finish buffering
buffering
finish buffering
buffering
finish buffering
buffering
finish buffering
buffering
finish buffering
buffering
finish buffering
buffering
finish buffering
buffering
finish buffering


In [8]:
pathlist=[]
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0372_07_00/') #7 61
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0372_07_01/') #7 59
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0374_11_00/') #11 116
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0374_11_01/') #11 108
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0374_11_02/') #11 154
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0374_11_03/') #11 125
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0378_13_00/') #13 40
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0379_11_00/') #11 67
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0380_11_00/') #11 121
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0384_11_01/') #11 100
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0384_11_02/') #11 62
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0384_11_03/') #11 86
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0384_11_04/') #11 150
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0385_11_00/') #11 64
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0386_11_00/') #11 107
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0477_11_00/') #11 155
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0857_11_00/') #11 54 
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0868_11_01/') #11 46
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0868_11_02/') #11 64
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0871_11_01/') #11 65
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0881_11_00/') #11 84
pathlist.append('E:/gopro/BLE_raw_bi_large/train/GOPR0884_11_00/') #11

In [9]:
import os
for name in pathlist:
    if os.path.exists(name) is False:
        os.makedirs(name)

In [10]:
pathlist=[]
pathlist.append(['E:/gopro/BLE_raw_bi_large/test/GOPR0384_11_00/',11])
pathlist.append(['E:/gopro/BLE_raw_bi_large/test/GOPR0384_11_05/',11])
pathlist.append(['E:/gopro/BLE_raw_bi_large/test/GOPR0385_11_01/',11])
pathlist.append(['E:/gopro/BLE_raw_bi_large/test/GOPR0396_11_00/',11])
pathlist.append(['E:/gopro/BLE_raw_bi_large/test/GOPR0410_11_00/',11])
pathlist.append(['E:/gopro/BLE_raw_bi_large/test/GOPR0854_11_00/',11])
pathlist.append(['E:/gopro/BLE_raw_bi_large/test/GOPR0862_11_00/',11])
pathlist.append(['E:/gopro/BLE_raw_bi_large/test/GOPR0868_11_00/',11])
pathlist.append(['E:/gopro/BLE_raw_bi_large/test/GOPR0869_11_00/',11])
pathlist.append(['E:/gopro/BLE_raw_bi_large/test/GOPR0871_11_00/',11])
pathlist.append(['E:/gopro/BLE_raw_bi_large/test/GOPR0881_11_01/',11])

In [11]:
import os
for name,frames in pathlist:
    if os.path.exists(name) is False:
        os.makedirs(name)